In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

EPOCH=10
BATCH_SIZE=64
LR=0.001
#load datasets
train_data=datasets.CIFAR10(root='/cifar10',train=True,
                           transform=transforms.ToTensor(),download=False)
test_data=datasets.CIFAR10(root='/cifar10',train=False,
                           transform=transforms.ToTensor(),download=False)
temp=train_data[1][0].numpy()
print(temp.shape)
temp=temp.transpose(1,2,0)
print(temp.shape)
plt.imshow(temp)
plt.show()
# train process
from torch.utils.data import DataLoader
train_loader=DataLoader(dataset=train_data, batch_size=BATCH_SIZE,shuffle=True)
test_loader=DataLoader(dataset=test_data, batch_size=BATCH_SIZE,shuffle=True)

model=torchvision.models.resnet50(pretrained=False)

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=LR)

device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model=model.to(device)

for epoch in range(EPOCH):
    for i, data in enumerate(train_loader):
        inputs, labels=data
        inputs, labels=inputs.to(device), labels.to(device)
        outputs=model(inputs)
        loss=criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch{} loss:{:.4f}',format(epoch+1, loss.item()))
    
torch.save(model, 'cifar10_resnet.pt')
print('cifar10_resnet.pt saved')

#test process
model=torch.load('cifar10_resnet.pt')
model.eval()
correct,total=0,0
for data in test_loader:
    inputs, labels=data
    inputs,labels=inputs.to(device),labels.to(device)
    outputs=model(inputs)
    _, predicted=torch.max(outputs.data,1)
    total=total+labels.size(0)
    correct=correct+(predicted==labels).sum().item()
    
print('accuracy: {:4f}%'.format(100.0*correct/total))

        
